In [1]:
import pyspark
import os
import json
import argparse

from dotenv import load_dotenv
from pathlib import Path
from pyspark.sql.types import StructType
from pyspark.sql.functions import to_timestamp,col,when

In [2]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

True

In [3]:
postgres_host = os.getenv('POSTGRES_CONTAINER_NAME')
postgres_dw_db = os.getenv('POSTGRES_DW_DB')
postgres_user = os.getenv('POSTGRES_USER')
postgres_password = os.getenv('POSTGRES_PASSWORD')

In [ ]:
postgres_host

In [4]:
sparkcontext = pyspark.SparkContext.getOrCreate(conf=(
        pyspark
        .SparkConf()
        .setAppName('Dibimbing')
        .setMaster('local')
        .set("spark.jars", "/opt/postgresql-42.2.18.jar")
    ))
sparkcontext.setLogLevel("WARN")

spark = pyspark.sql.SparkSession(sparkcontext.getOrCreate())

# JDBC

In [5]:
jdbc_url = f'jdbc:postgresql://{postgres_host}/{postgres_dw_db}'
jdbc_properties = {
    'user': postgres_user,
    'password': postgres_password,
    'driver': 'org.postgresql.Driver',
    'stringtype': 'unspecified'
}

In [6]:
retail_df = spark.read.jdbc(
    jdbc_url,
    'public.retail',
    properties=jdbc_properties
)
retail_df.show()

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|invoiceno|stockcode|         description|quantity|invoicedate|unitprice|customerid|       country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6| 2010-12-01|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6| 2010-12-01|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8| 2010-12-01|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6| 2010-12-01|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6| 2010-12-01|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2| 2010-12-01|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6| 2010-12-01|     4.25|     17850|United Kingdom|


In [6]:
#read csv data
df_csv = spark.read.format("csv").option("header","true")\
    .load("/resources/data/owid-covid-data.csv")
#df_csv.show(10)

## Aggregate data using SQL

In [7]:
# Register the DataFrame as a SQL temporary view
df_csv.createOrReplaceTempView("owid_covid")

In [8]:
sqlDF = spark.sql("""
                   SELECT location, count(total_cases) as total_cases, count(total_deaths) as total_deaths
                  FROM owid_covid
                  group by location
                  order by total_cases desc
                  """)
#sqlDF.show()

## Write to postgresql

In [10]:
sqlDF.write.mode("overwrite").jdbc(
        jdbc_url,
        'public.count_owid_covid',
        properties=jdbc_properties
    )

In [14]:
spark.read.jdbc(
        jdbc_url,
        'public.count_owid_covid',
        properties=jdbc_properties
    ).show()

+--------------+-----------+------------+
|      location|total_cases|total_deaths|
+--------------+-----------+------------+
|        Europe|        209|         209|
|       Germany|        209|         209|
|         China|        209|         208|
|European Union|        209|         209|
|   High income|        209|         209|
|          Asia|        209|         208|
|        France|        206|         199|
|        Canada|        206|         199|
|     Australia|        206|         200|
|      Cambodia|        205|         147|
|       Finland|        205|         200|
|         India|        205|         199|
|       Belgium|        204|         199|
|       Iceland|        204|         198|
|        Africa|        203|         200|
|         Egypt|        203|         199|
|          Iran|        202|         202|
|          Iraq|        201|         200|
|   Afghanistan|        201|         197|
|        Greece|        201|         199|
+--------------+-----------+------